In [1]:
import pandas as pd
import openpyxl
from pandas import DataFrame
from sklearn import tree
import pickle
import json
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [2]:
input_df = pd.read_csv('data.csv')
length = input_df.shape[0]

In [3]:
f2 = open('nlp.pkl','rb')
dict_nlp_eve = pickle.load(f2)
f2.close()

In [4]:
all_employees=[]

In [5]:
df = pd.read_csv('CCMLEmployeeData.csv')
copydf = df.copy()

In [6]:
inst = open('Extract_Domain.pkl','rb')                
Domain_extract = pickle.load(inst)
inst2 = open("Extract_Event.pkl","rb")
Event_extract = pickle.load(inst2)

In [7]:
event_list=[]
for x in range(0,length):
    event_list.append((input_df.iloc[x,0]).upper())
print(event_list)

['CODEBOOST CODEATHON IS LIVE NOW!"', 'NATIONAL WEBINAR ON BANKS, NBFCS & CAPITAL MARKETS', 'FREE COACHING FOR CAT/NMAT/CET/B.E/B.TECH', 'SUMMER INTERNSHIP OPPORTUNITY: WORK AS A TEACHER FOR GRE STUDENTS', 'REGISTER FOR A WEBINAR REGARDING MASTERS IN THE USA', 'HOW TO SCORE 340/340 IN THE GRE? JOIN US IN THIS FREE WEBINAR TO FIND OUT', 'ATTEND OUR HIGHER EDUCATION FAIR AND FIND THE RIGHT COLLEGE FOR YOU', '7 TOP-RANKED IRELAND UNIVERSITIES - ONLINE WEBINARS!', 'LOCKDOWN SPECIAL COURSES ON YDEMI. 22 HOURS LEFT!', 'BUILD YOUR RESUME BY JOINING US FOR OUR SUMMER INTERNSHIP PROGRAM ', 'INTRODUCING THE MANAGEMENT 2.0 HACKATHON', 'BLOCKCHAIN TALKS: SPEECH ON THE BLOCKCHAIN LIFE CYCLE', 'FREE BLOCKCHAIN WORKSHOP BY GREENLAWNS', 'ATTEND A PAID BLOCKCHAIN WORKSHOP AND FAIR', 'WORKSHOPS AND FAIRS: THE ANNUAL BLOCKCHAIN FAIR IS OUT NOW. REGISTER FOR THE THE WORKSHOP TODAY', 'JOIN US FOR A BLOCKCHAIN WORKSHOP AND LEARN THE BEAUTY OF BLOCKCHAINS', 'COMPLIMENTARY WORKSHOP/ FAIR ON BLOCKCHAIN. SIGN U

In [8]:
def domain_extractor(t):
    words=[]
    flag_domain = 0
    word = t.split(',')
    for m in word:
        tmp = m.split()
        for x in tmp:
            words.append(x)
    DE= []
    for x in words:
        try:
            DE.append(Domain_extract[x])
            flag_domain = 1
        except:
            pass
        
    if True:
        for x in words:
            for y in words:
                try:
                    dummy = x+' '+y
                    DE.append(Domain_extract[dummy])
                except:
                    pass  
    DE = list(dict.fromkeys(DE))
    return DE

In [9]:
def domain_extractor_ml(t):
    DE_ml=[]
    nlp_df = DataFrame(list(Domain_extract.items()),columns = ['txt','Domain'])
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    clf_svm = svm.SVC(kernel='rbf')
    clf_svm.fit(train_vectors,nlp_df.Domain)
    pred = clf_svm.predict(temp1)[0]
    DE_ml.append(pred)
  
    clf_tree = DecisionTreeClassifier()
    clf_tree.fit(train_vectors,nlp_df.Domain)
    pred = clf_tree.predict(temp1)[0]
    DE_ml.append(pred)
    
    clf_bay = GaussianNB()
    clf_bay.fit(train_vectors.toarray(),nlp_df.Domain)
    pred = clf_bay.predict(temp1.toarray())[0]
    DE_ml.append(pred)
    
    DE_ml = list(dict.fromkeys(DE_ml))
    return DE_ml

In [10]:
def event_extractor(t):
    words=t.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    words = [re_punc.sub('', x) for x in words]
    EE= []
    for x in words:
        try:            
            EE.append(Event_extract[x])
        except:
            pass

    return EE

In [11]:
def event_extractor_ml(t):
    EE_ml=[]
    nlp_df = DataFrame(list(dict_nlp_eve.items()),columns = ['Event','txt']) 
    vectorizer = CountVectorizer(stop_words='english',binary=True)
    train_vectors = vectorizer.fit_transform(nlp_df.txt)
    temp1 = vectorizer.transform([t])
    
    clf_bay = GaussianNB()
    clf_bay.fit(train_vectors.toarray(),nlp_df.Event)
    pred = clf_bay.predict(temp1.toarray())[0]
    EE_ml.append(pred)
    
    return EE_ml

In [12]:
def employees_event(DE,EE):
    employees=[]
    for x in DE:
        temp1 = copydf[copydf['Domain']==x].copy()
        for y in EE:
            temp2 = temp1[temp1['Event1']==y].copy()
            temp3 = temp1[temp1['Event2']==y].copy()
            
            try:
                for z in temp2.Name:
                    employees.append(z)
                for z in temp3.Name:
                    employees.append(z)
            except:
                pass

    return employees

In [13]:

for t in event_list:
    DE = domain_extractor(t)
    DE_ml = domain_extractor_ml(t)
    EE =  event_extractor(t)
    EE_ml = event_extractor_ml(t)
    for x in DE_ml:
        if x not in DE:
            DE.append(x)
    DE = list(dict.fromkeys(DE))
    employees = employees_event(DE,EE)
    if not employees:
          employees.append('None')
    all_employees.append(employees)

In [14]:
dict1={}
for totalppl,event in zip(all_employees,event_list):
    strx = str(totalppl)[1:-1]
    strxx = strx.replace("'", "")
    dict1[event]=strxx

In [15]:
dfx = DataFrame(list(dict1.items()),columns = ['Event Name','Employee Names']) 
dfx

,Event Name,Employee Names
0,"CODEBOOST CODEATHON IS LIVE NOW!""",Laura Hurst
1,"NATIONAL WEBINAR ON BANKS, NBFCS & CAPITAL MAR...","Kenneth Martinez, Melissa Harris, Mrs. Brenda ..."
2,FREE COACHING FOR CAT/NMAT/CET/B.E/B.TECH,None
3,SUMMER INTERNSHIP OPPORTUNITY: WORK AS A TEACH...,"Melinda West, Damon Lane"
4,REGISTER FOR A WEBINAR REGARDING MASTERS IN TH...,None
5,HOW TO SCORE 340/340 IN THE GRE? JOIN US IN TH...,"Kenneth Martinez, Melissa Harris, Mrs. Brenda ..."
6,ATTEND OUR HIGHER EDUCATION FAIR AND FIND THE ...,None
7,7 TOP-RANKED IRELAND UNIVERSITIES - ONLINE WEB...,"Kenneth Martinez, Melissa Harris, Mrs. Brenda ..."
8,LOCKDOWN SPECIAL COURSES ON YDEMI. 22 HOURS LEFT!,"Jennifer Merritt, Robert Ramirez, Michelle Mil..."
9,BUILD YOUR RESUME BY JOINING US FOR OUR SUMMER...,"Melinda West, Damon Lane"


In [16]:
dfx.to_excel('result.xlsx', index = None, header=True)
print('Thank you for using Event Recommrndation System!\nCheck Your Excel Sheet for recommendations')

Thank you for using Event Recommrndation System!
Check Your Excel Sheet for recommendations
